## Buildkite Demo

### Prequesites

In [2]:
import os
import time
from dotenv import load_dotenv

load_dotenv()
PERSONAL_ACCESS_TOKEN: str = os.environ.get("BUILDKITE_ACCESS_TOKEN")
ORGANISATION: str = "bmwgroup"

### Auth

In [3]:
from pybuildkite.buildkite import Buildkite, BuildState

buildkite: Buildkite = Buildkite()
buildkite.set_access_token(PERSONAL_ACCESS_TOKEN)

### Organisation

In [4]:
from typing import Any, Dict

organisation_infos: Dict[str, Any] = buildkite.organizations().get_org(ORGANISATION)
print(organisation_infos)

{'id': '0192f6f2-2d4d-4127-acf6-877aeaf204a0', 'graphql_id': 'T3JnYW5pemF0aW9uLS0tMDE5MmY2ZjItMmQ0ZC00MTI3LWFjZjYtODc3YWVhZjIwNGEw', 'url': 'https://api.buildkite.com/v2/organizations/bmwgroup', 'web_url': 'https://buildkite.com/bmwgroup', 'name': 'bmwgroup', 'slug': 'bmwgroup', 'agents_url': 'https://api.buildkite.com/v2/organizations/bmwgroup/agents', 'emojis_url': 'https://api.buildkite.com/v2/organizations/bmwgroup/emojis', 'created_at': '2024-11-04T11:31:59.436Z', 'pipelines_url': 'https://api.buildkite.com/v2/organizations/bmwgroup/pipelines'}


### Pipelines

In [10]:
from multiprocessing.connection import Client
from pybuildkite.pipelines import Pipelines

pipelines: Pipelines = buildkite.pipelines().list_pipelines(ORGANISATION)

print(pipelines)

for pipe in pipelines:
    print(pipe)

pipeline_path: str = pipelines.path
pipeline_client: Client = pipelines.client

print(f"Path: {pipeline_path}\nClient: {pipeline_client}")


[{'id': '01936deb-a4db-45f2-b601-1b421ceccb6d', 'graphql_id': 'UGlwZWxpbmUtLS0wMTkzNmRlYi1hNGRiLTQ1ZjItYjYwMS0xYjQyMWNlY2NiNmQ=', 'url': 'https://api.buildkite.com/v2/organizations/bmwgroup/pipelines/bash-parallel-load-testing', 'web_url': 'https://buildkite.com/bmwgroup/bash-parallel-load-testing', 'name': 'Bash Parallel - Load Testing', 'description': 'Load testing with spot and parallelism', 'slug': 'bash-parallel-load-testing', 'repository': 'https://github.com/buildkite-demo-org/bash-parallel-load-testing.git', 'cluster_id': 'adc7fdf8-aadc-4bd1-9dea-56476c2a8c88', 'pipeline_template_uuid': None, 'branch_configuration': None, 'default_branch': 'main', 'skip_queued_branch_builds': False, 'skip_queued_branch_builds_filter': None, 'cancel_running_branch_builds': False, 'cancel_running_branch_builds_filter': None, 'allow_rebuilds': True, 'provider': {'id': 'github', 'settings': {'build_branches': True, 'build_merge_group_checks_requested': False, 'build_pull_request_base_branch_changed

KeyboardInterrupt: 

### Build Info

In [6]:
from pybuildkite.builds import Builds

pipeline: str = "bazel-build"

state_running: BuildState = BuildState.RUNNING
scheduled: BuildState = BuildState.SCHEDULED
canceled: BuildState = BuildState.CANCELED
failed: BuildState = BuildState.FAILED
passed: BuildState = BuildState.PASSED

running_or_scheduled: Builds = buildkite.builds().list_all_for_pipeline(ORGANISATION, pipeline,
                                                                        states=[state_running, scheduled])
print(f"Running or scheduled builds: {[build["message"] for build in running_or_scheduled]}")

canceled_or_failed: Builds = buildkite.builds().list_all_for_pipeline(ORGANISATION, pipeline, states=[canceled, failed])
print(f"Canceled or failed builds: {[build["message"] for build in canceled_or_failed]}")

passed_builds: Builds = buildkite.builds().list_all_for_pipeline(ORGANISATION, pipeline, states=[passed])
print(f"Canceled or passed builds: {[build["message"] for build in passed_builds]}")

Running or scheduled builds: []


KeyboardInterrupt: 

### Create Build

In [29]:
pipeline: str = "bazel-build"

build_info = buildkite.builds().create_build(ORGANISATION, pipeline, 'HEAD', 'master',
                                             clean_checkout=True, message="Build via Python Client!")

print(f"Build sent!\nBuild Info:{build_info}")
CREATED_AT = build_info["created_at"]

Build sent!
Build Info:{'id': '01936df2-d713-4c81-9dbe-9485e474655e', 'graphql_id': 'QnVpbGQtLS0wMTkzNmRmMi1kNzEzLTRjODEtOWRiZS05NDg1ZTQ3NDY1NWU=', 'url': 'https://api.buildkite.com/v2/organizations/bmwgroup/pipelines/bazel-build/builds/41', 'web_url': 'https://buildkite.com/bmwgroup/bazel-build/builds/41', 'number': 41, 'state': 'scheduled', 'cancel_reason': None, 'blocked': False, 'blocked_state': '', 'message': 'Build via Python Client!', 'commit': 'HEAD', 'branch': 'master', 'tag': None, 'env': {}, 'source': 'api', 'author': None, 'creator': {'id': '0193004b-dc06-4728-9a4f-d8e37ca04f09', 'graphql_id': 'VXNlci0tLTAxOTMwMDRiLWRjMDYtNDcyOC05YTRmLWQ4ZTM3Y2EwNGYwOQ==', 'name': 'Oliver Königsberger', 'email': 'oliver@koenigsberger.de', 'avatar_url': 'https://avatars.githubusercontent.com/u/102287892?v=4', 'created_at': '2024-11-06T07:06:31.813Z'}, 'created_at': '2024-11-27T14:07:31.624Z', 'scheduled_at': '2024-11-27T14:07:31.523Z', 'started_at': None, 'finished_at': None, 'meta_data': {}

### Build Response

In [30]:
from typing import Optional
from pybuildkite.client import Response

BUILD_RESULT: Optional[Any] = None

while True:
    builds_response: Response = buildkite.builds().list_all(page=1, with_pagination=True)

    while builds_response.next_page:
        print("Searching for build result")
        for build in builds_response.body:
            if build["created_at"] == CREATED_AT:
                BUILD_RESULT = build
                break

        builds_response = buildkite.builds().list_all(page=builds_response.next_page, with_pagination=True)

    if BUILD_RESULT is not None and BUILD_RESULT["state"] not in ["scheduled", "running"]:
        break
    else:
        time.sleep(1)

print(f"Buildresult of \"{BUILD_RESULT["message"]}\" is \"{BUILD_RESULT["state"]}\"")
print(BUILD_RESULT)


Searching for build result
Searching for build result
Searching for build result
Searching for build result
Searching for build result
Searching for build result
Buildresult of "Build via Python Client!" was "passed"
{'id': '01936df2-d713-4c81-9dbe-9485e474655e', 'graphql_id': 'QnVpbGQtLS0wMTkzNmRmMi1kNzEzLTRjODEtOWRiZS05NDg1ZTQ3NDY1NWU=', 'url': 'https://api.buildkite.com/v2/organizations/bmwgroup/pipelines/bazel-build/builds/41', 'web_url': 'https://buildkite.com/bmwgroup/bazel-build/builds/41', 'number': 41, 'state': 'passed', 'cancel_reason': None, 'blocked': False, 'blocked_state': '', 'message': 'Build via Python Client!', 'commit': '2aac26f48791596b6c1dae593ef5d22d48d1ea22', 'branch': 'master', 'tag': None, 'env': {}, 'source': 'api', 'author': None, 'creator': {'id': '0193004b-dc06-4728-9a4f-d8e37ca04f09', 'graphql_id': 'VXNlci0tLTAxOTMwMDRiLWRjMDYtNDcyOC05YTRmLWQ4ZTM3Y2EwNGYwOQ==', 'name': 'Oliver Königsberger', 'email': 'oliver@koenigsberger.de', 'avatar_url': 'https://avatar

In [38]:
import json
from pathlib import Path

with open(Path.cwd().joinpath("current_build_result.json"), "w") as file:
    json.dump(BUILD_RESULT, file, indent=2)
